## 環境構築

In [ ]:
import logging
import os
import subprocess
import sys

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)

NVIDIA_SMI = subprocess.run(["nvidia-smi"], capture_output=True, text=True).stdout
logging.info(NVIDIA_SMI)
logging.info(f"Python {sys.version}")

In [ ]:
# 1) PyTorchとTransformersのインストール
%pip install torch==2.4.0

# 2) vLLMのインストール
%pip install vllm==0.6.3

# 3) Flash Attentionのインストール
# 2.8.3はundefined symbolエラーが発生するため2.7.3をインストール
# https://github.com/Dao-AILab/flash-attention/issues/1832
# %pip install "https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu12torch2.6cxx11abiFALSE-cp312-cp312-linux_x86_64.whl" --no-build-isolation
%pip install flash-attn --no-build-isolation

In [ ]:
%cd /workspaces/tinyzero/TinyZero
%pip install -e .

In [ ]:
%pip install wandb

## データセット構築

```sh
python ./examples/data_preprocess/countdown.py --local_dir countdown
```

In [ ]:
import re
import os
from datasets import Dataset, load_dataset
from random import randint, seed, choice
from typing import List, Tuple
from tqdm import tqdm
from verl.utils.hdfs_io import copy, makedirs
import argparse

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--local_dir', default='~/data/countdown')
parser.add_argument('--hdfs_dir', default=None)
parser.add_argument('--num_samples', type=int, default=100000)
parser.add_argument('--num_operands', type=int, default=6)
parser.add_argument('--max_target', type=int, default=1000)
parser.add_argument('--min_number', type=int, default=1)
parser.add_argument('--max_number', type=int, default=100)
parser.add_argument('--train_size', type=int, default=327680)
parser.add_argument('--test_size', type=int, default=1024)
parser.add_argument('--template_type', type=str, default='base')

args = parser.parse_args(["--local_dir", "countdown"])

data_source = 'countdown'
TRAIN_SIZE = args.train_size
TEST_SIZE = args.test_size

TRAIN_SIZE, TEST_SIZE

In [ ]:
# https://huggingface.co/datasets/Jiayi-Pan/Countdown-Tasks-3to4
raw_dataset = load_dataset('Jiayi-Pan/Countdown-Tasks-3to4', split='train')
len(raw_dataset)

In [ ]:
assert len(raw_dataset) > TRAIN_SIZE + TEST_SIZE
train_dataset = raw_dataset.select(range(TRAIN_SIZE))
test_dataset = raw_dataset.select(range(TRAIN_SIZE, TRAIN_SIZE + TEST_SIZE))

len(train_dataset), len(test_dataset)

In [ ]:
train_dataset[0]

In [ ]:
def make_prefix(dp, template_type):
    """
    データポイントからプロンプトのプレフィックスを作成する

    Args:
        dp (dict): targetとnumsを含むデータポイント
        template_type (str): 使用するテンプレートの種類
            - base: 一般的なベースモデル用
            - qwen-instruct: Qwen Instructモデル用

    Returns:
        str: プロンプトのプレフィックス
    """
    target = dp['target']
    numbers = dp['nums']

    # ベースモデルの場合
    if template_type == 'base':
        # User: {numbers} を使って {target} になる数式を作ってください。
        # 四則演算（+, -, *, /）が使え、各数字は一度だけ使えます。
        # 思考過程は <think> </think> タグで示し、最終的な答えは <answer> </answer> タグで
        # 返してください。例: <answer> (1 + 2) / 3 </answer>。
        # Assistant: 順を追って解いていきます。<think>
        prefix = f"""A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
User: Using the numbers {numbers}, create an equation that equals {target}. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.
Assistant: Let me solve this step by step.
<think>"""

    # Qwen Instructモデルの場合
    elif template_type == 'qwen-instruct':

        # <|im_start|>system
        # あなたは有能なアシスタントです。まず頭の中で推論プロセスを考え、その後ユーザーに答えを伝えます。
        # <|im_end|>
        # <|im_start|>user
        # {numbers} を使って {target} になる数式を作ってください。
        # 四則演算（+, -, *, /）が使え、各数字は一度だけ使えます。
        # 思考過程は <think> </think> タグで示し、最終的な答えは
        #  <answer> </answer> タグで返してください。
        # 例: <answer> (1 + 2) / 3 </answer>。
        # <|im_end|>
        # <|im_start|>assistant
        # 順を追って解いていきます。
        # <think>
        prefix = f"""<|im_start|>system\nYou are a helpful assistant. You first thinks about the reasoning process in the mind and then provides the user with the answer.<|im_end|>\n<|im_start|>user\n Using the numbers {numbers}, create an equation that equals {target}. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.<|im_end|>\n<|im_start|>assistant\nLet me solve this step by step.\n<think>"""
    return prefix

make_prefix(train_dataset[0], 'base')

In [ ]:
def make_map_fn(split):
    """
    データセットの前処理に使用するマップ関数を作成する

    Args:
        split (str): データセットの分割名（例: 'train', 'test'）
    Returns:
        function: データポイントを処理するマップ関数
    """

    def process_fn(example, idx):
        """
        データポイントを前処理する

        Args:
            example (dict): データポイント
            idx (int): データポイントのインデックス
        Returns:
            dict: 前処理されたデータポイント
        """

        # データポイントからプロンプトを作成
        question = make_prefix(example, template_type=args.template_type)

        solution = {
            "target": example['target'],
            "numbers": example['nums']
        }

        # 前処理したデータを辞書形式で返す
        data = {
            "data_source": data_source,
            "prompt": [{
                "role": "user",
                "content": question,
            }],
            "ability": "math",
            "reward_model": {
                "style": "rule",
                "ground_truth": solution
            },
            "extra_info": {
                'split': split,
                'index': idx,
            }
        }
        return data
    return process_fn

# 検証
make_map_fn("train")(train_dataset[0], 0)

In [ ]:
# データセットの前処理とParquet形式での保存

local_dir = args.local_dir

if not os.path.exists(os.path.join(local_dir, "train.parquet")):
    train_dataset = train_dataset.map(function=make_map_fn('train'), with_indices=True)
    train_dataset.to_parquet(os.path.join(local_dir, 'train.parquet'))

if not os.path.exists(os.path.join(local_dir, "test.parquet")):
    test_dataset = test_dataset.map(function=make_map_fn('test'), with_indices=True)
    test_dataset.to_parquet(os.path.join(local_dir, 'test.parquet'))

In [ ]:
# Hadoop File System (HDFS) にデータをコピー
# 分散処理用

hdfs_dir = args.hdfs_dir

if hdfs_dir is not None:
    makedirs(hdfs_dir)
    copy(src=local_dir, dst=hdfs_dir) 

## 訓練

```sh
N_GPUS=1 BASE_MODEL="qwen/qwen2.5-0.5b" DATA_DIR="countdown" ROLLOUT_TP_SIZE=1 VLLM_ATTENTION_BACKEND="XFORMERS" bash ./scripts/train_tiny_zero.sh
```

In [ ]:
from verl import DataProto
import torch
from verl.utils.reward_score import gsm8k, math, multiply, countdown
from verl.trainer.ppo.ray_trainer import RayPPOTrainer
import ray
import hydra

In [ ]:
# コマンドラインの環境変数を変数化

BASE_MODEL="qwen/qwen2.5-0.5b"
DATA_DIR = "countdown"
EXPERIMENT_NAME = "countdown_ppo_demo"
N_GPUS = 1
ROLLOUT_TP_SIZE = 1
VLLM_ATTENTION_BACKEND = "XFORMERS"

In [ ]:
# PPOトレーナーのデフォルト設定を読み込み

from omegaconf import OmegaConf

# TinyZero/verl/trainer/config/ppo_trainer.yaml
ppo_trainer_yaml = """
data:
  tokenizer: null
  train_files: ~/data/rlhf/gsm8k/train.parquet
  val_files: ~/data/rlhf/gsm8k/test.parquet
  prompt_key: prompt
  max_prompt_length: 512
  max_response_length: 512
  train_batch_size: 1024
  val_batch_size: 1312
  return_raw_input_ids: False  # This should be set to true when the tokenizer between policy and rm differs
  return_raw_chat: False

actor_rollout_ref:
  hybrid_engine: True
  model:
    path: ~/models/deepseek-llm-7b-chat
    external_lib: null
    override_config: { }
    enable_gradient_checkpointing: False
    use_remove_padding: False
  actor:
    strategy: fsdp  # This is for backward-compatibility
    ppo_mini_batch_size: 256
    ppo_micro_batch_size: 64
    use_dynamic_bsz: False
    ppo_max_token_len_per_gpu: 16384 # n * ${data.max_prompt_length} + ${data.max_response_length}
    grad_clip: 1.0
    clip_ratio: 0.2
    entropy_coeff: 0.001
    use_kl_loss: False # True for GRPO
    kl_loss_coef: 0.001 # for grpo
    kl_loss_type: low_var_kl # for grpo
    ppo_epochs: 1
    shuffle: False
    ulysses_sequence_parallel_size: 1 # sp size
    optim:
      lr: 1e-6
      lr_warmup_steps_ratio: 0.  # the total steps will be injected during runtime
      min_lr_ratio: null   # only useful for warmup with cosine
      warmup_style: constant  # select from constant/cosine
      total_training_steps: -1  # must be override by program
    fsdp_config:
      wrap_policy:
        # transformer_layer_cls_to_wrap: None
        min_num_params: 0
      param_offload: False
      grad_offload: False
      optimizer_offload: False
      fsdp_size: -1
  ref:
    fsdp_config:
      param_offload: False
      wrap_policy:
        # transformer_layer_cls_to_wrap: None
        min_num_params: 0
      fsdp_size: -1
    log_prob_micro_batch_size: 128
    log_prob_use_dynamic_bsz: ${actor_rollout_ref.actor.use_dynamic_bsz}
    log_prob_max_token_len_per_gpu: ${actor_rollout_ref.actor.ppo_max_token_len_per_gpu}
    ulysses_sequence_parallel_size: ${actor_rollout_ref.actor.ulysses_sequence_parallel_size} # sp size
  rollout:
    name: vllm
    temperature: 1.0
    top_k: -1 # 0 for hf rollout, -1 for vllm rollout
    top_p: 1
    prompt_length: ${data.max_prompt_length}  # not use for opensource
    response_length: ${data.max_response_length}
    # for vllm rollout
    dtype: bfloat16 # should align with FSDP
    gpu_memory_utilization: 0.5
    ignore_eos: False
    enforce_eager: True
    free_cache_engine: True
    load_format: dummy_dtensor
    tensor_model_parallel_size: 2
    max_num_batched_tokens: 8192
    max_num_seqs: 1024
    log_prob_micro_batch_size: 128
    log_prob_use_dynamic_bsz: ${actor_rollout_ref.actor.use_dynamic_bsz}
    log_prob_max_token_len_per_gpu: ${actor_rollout_ref.actor.ppo_max_token_len_per_gpu}
    # for hf rollout
    do_sample: True
    # number of responses (i.e. num sample times)
    n: 1 # > 1 for grpo

critic:
  strategy: fsdp
  optim:
    lr: 1e-5
    lr_warmup_steps_ratio: 0.  # the total steps will be injected during runtime
    min_lr_ratio: null   # only useful for warmup with cosine
    warmup_style: constant  # select from constant/cosine
    total_training_steps: -1  # must be override by program
  model:
    path: ~/models/deepseek-llm-7b-chat
    tokenizer_path: ${actor_rollout_ref.model.path}
    override_config: { }
    external_lib: ${actor_rollout_ref.model.external_lib}
    enable_gradient_checkpointing: False
    use_remove_padding: False
    fsdp_config:
      param_offload: False
      grad_offload: False
      optimizer_offload: False
      wrap_policy:
        # transformer_layer_cls_to_wrap: None
        min_num_params: 0
      fsdp_size: -1
  ppo_mini_batch_size: ${actor_rollout_ref.actor.ppo_mini_batch_size}
  ppo_micro_batch_size: 64
  forward_micro_batch_size: ${critic.ppo_micro_batch_size}
  use_dynamic_bsz: ${actor_rollout_ref.actor.use_dynamic_bsz}
  ppo_max_token_len_per_gpu: 32768 # (${actor_rollout_ref.actor.ppo_max_token_len_per_gpu}) * 2
  forward_max_token_len_per_gpu: ${critic.ppo_max_token_len_per_gpu}
  ulysses_sequence_parallel_size: 1 # sp size
  ppo_epochs: ${actor_rollout_ref.actor.ppo_epochs}
  shuffle: ${actor_rollout_ref.actor.shuffle}
  grad_clip: 1.0
  cliprange_value: 0.5

reward_model:
  enable: False
  strategy: fsdp
  model:
    input_tokenizer: ${actor_rollout_ref.model.path}  # set this to null if the chat template is identical
    path: ~/models/FsfairX-LLaMA3-RM-v0.1
    external_lib: ${actor_rollout_ref.model.external_lib}
    use_remove_padding: False
    fsdp_config:
      min_num_params: 0
      param_offload: False
  micro_batch_size: 64
  max_length: null
  ulysses_sequence_parallel_size: 1 # sp size
  use_dynamic_bsz: ${critic.use_dynamic_bsz}
  forward_max_token_len_per_gpu: ${critic.forward_max_token_len_per_gpu}

algorithm:
  gamma: 1.0
  lam: 1.0
  adv_estimator: gae
  kl_penalty: kl  # how to estimate kl divergence
  kl_ctrl:
    type: fixed
    kl_coef: 0.001

trainer:
  total_epochs: 30
  total_training_steps: null
  project_name: verl_examples
  experiment_name: gsm8k
  logger: [ 'console', 'wandb' ]
  nnodes: 1
  n_gpus_per_node: 8
  save_freq: -1
  test_freq: -1
  critic_warmup: 0
  default_hdfs_dir: ~/experiments/gsm8k/ppo/${trainer.experiment_name}
  default_local_dir: checkpoints/${trainer.project_name}/${trainer.experiment_name}
"""

ppo_trainer_config = OmegaConf.create(ppo_trainer_yaml)

In [ ]:
# PPOトレーニングのデフォルト設定を上書き

# TinyZero/scripts/train_tiny_zero.shより抜粋
# python3 -m verl.trainer.main_ppo \
# data.train_files=$DATA_DIR/train.parquet \
# data.val_files=$DATA_DIR/test.parquet \
# data.train_batch_size=32 \
# data.val_batch_size=32 \
# data.max_prompt_length=256 \
# data.max_response_length=1024 \
# actor_rollout_ref.model.path=$BASE_MODEL \
# actor_rollout_ref.model.use_remove_padding=True \
# actor_rollout_ref.model.enable_gradient_checkpointing=True \
# actor_rollout_ref.actor.use_dynamic_bsz=True \
# actor_rollout_ref.actor.optim.lr=1e-6 \
# actor_rollout_ref.actor.ppo_mini_batch_size=4 \
# actor_rollout_ref.actor.ppo_micro_batch_size=4 \
# actor_rollout_ref.rollout.log_prob_micro_batch_size=1 \
# actor_rollout_ref.rollout.tensor_model_parallel_size=$ROLLOUT_TP_SIZE \
# actor_rollout_ref.rollout.gpu_memory_utilization=0.1 \
# actor_rollout_ref.ref.log_prob_micro_batch_size=1 \
# critic.optim.lr=1e-5 \
# critic.model.path=$BASE_MODEL \
# critic.ppo_micro_batch_size=1 \
# algorithm.kl_ctrl.kl_coef=0.001 \
# trainer.logger=[] \
# +trainer.val_before_train=False \
# trainer.default_hdfs_dir=null \
# trainer.n_gpus_per_node=$N_GPUS \
# trainer.nnodes=1 \
# trainer.save_freq=100 \
# trainer.test_freq=100 \
# trainer.project_name=TinyZero \
# trainer.experiment_name=$EXPERIMENT_NAME \
# trainer.total_epochs=15 2>&1 | tee verl_demo.log

ppo_trainer_config.data.train_files = os.path.join(DATA_DIR, "train.parquet")
ppo_trainer_config.data.val_files = os.path.join(DATA_DIR, "test.parquet")
ppo_trainer_config.data.train_batch_size = 32
ppo_trainer_config.data.val_batch_size = 32
ppo_trainer_config.data.max_prompt_length = 256
ppo_trainer_config.data.max_response_length = 1024
ppo_trainer_config.actor_rollout_ref.model.path = BASE_MODEL
ppo_trainer_config.actor_rollout_ref.model.use_remove_padding = True
ppo_trainer_config.actor_rollout_ref.model.enable_gradient_checkpointing = True
ppo_trainer_config.actor_rollout_ref.actor.use_dynamic_bsz = True
ppo_trainer_config.actor_rollout_ref.actor.optim.lr = 1e-6
ppo_trainer_config.actor_rollout_ref.actor.ppo_mini_batch_size = 4
ppo_trainer_config.actor_rollout_ref.actor.ppo_micro_batch_size = 4
ppo_trainer_config.actor_rollout_ref.rollout.log_prob_micro_batch_size = 1
ppo_trainer_config.actor_rollout_ref.rollout.tensor_model_parallel_size = ROLLOUT_TP_SIZE
ppo_trainer_config.actor_rollout_ref.rollout.gpu_memory_utilization = 0.1
ppo_trainer_config.actor_rollout_ref.ref.log_prob_micro_batch_size = 1
ppo_trainer_config.critic.optim.lr = 1e-5
ppo_trainer_config.critic.model.path = BASE_MODEL
ppo_trainer_config.critic.ppo_micro_batch_size = 1
ppo_trainer_config.algorithm.kl_ctrl.kl_coef = 0.001
ppo_trainer_config.trainer.logger = []
ppo_trainer_config.trainer.val_before_train = False
ppo_trainer_config.trainer.default_hdfs_dir = None
ppo_trainer_config.trainer.n_gpus_per_node = N_GPUS
ppo_trainer_config.trainer.nnodes = 1
ppo_trainer_config.trainer.save_freq = 100
ppo_trainer_config.trainer.test_freq = 100
ppo_trainer_config.trainer.project_name = "TinyZero"
ppo_trainer_config.trainer.experiment_name = EXPERIMENT_NAME

In [ ]:
def _select_rm_score_fn(data_source):
    if data_source == 'openai/gsm8k':
        return gsm8k.compute_score
    elif data_source == 'lighteval/MATH':
        return math.compute_score
    elif "multiply" in data_source or "arithmetic" in data_source:
        return multiply.compute_score
    elif "countdown" in data_source:
        return countdown.compute_score
    else:
        raise NotImplementedError

In [ ]:
class RewardManager():
    """The reward manager.
    """

    def __init__(self, tokenizer, num_examine) -> None:
        self.tokenizer = tokenizer
        self.num_examine = num_examine  # the number of batches of decoded responses to print to the console

    def __call__(self, data: DataProto):
        """We will expand this function gradually based on the available datasets"""

        # If there is rm score, we directly return rm score. Otherwise, we compute via rm_score_fn
        if 'rm_scores' in data.batch.keys():
            return data.batch['rm_scores']

        reward_tensor = torch.zeros_like(data.batch['responses'], dtype=torch.float32)

        already_print_data_sources = {}

        for i in range(len(data)):
            data_item = data[i]  # DataProtoItem

            prompt_ids = data_item.batch['prompts']

            prompt_length = prompt_ids.shape[-1]

            valid_prompt_length = data_item.batch['attention_mask'][:prompt_length].sum()
            valid_prompt_ids = prompt_ids[-valid_prompt_length:]

            response_ids = data_item.batch['responses']
            valid_response_length = data_item.batch['attention_mask'][prompt_length:].sum()
            valid_response_ids = response_ids[:valid_response_length]

            # decode
            sequences = torch.cat((valid_prompt_ids, valid_response_ids))
            sequences_str = self.tokenizer.decode(sequences)

            ground_truth = data_item.non_tensor_batch['reward_model']['ground_truth']

            # select rm_score
            data_source = data_item.non_tensor_batch['data_source']
            compute_score_fn = _select_rm_score_fn(data_source)

            score = compute_score_fn(solution_str=sequences_str, ground_truth=ground_truth)
            reward_tensor[i, valid_response_length - 1] = score

            if data_source not in already_print_data_sources:
                already_print_data_sources[data_source] = 0

            if already_print_data_sources[data_source] < self.num_examine:
                already_print_data_sources[data_source] += 1
                print(sequences_str)

        return reward_tensor

In [ ]:
# @ray.remote
def main_task(config):
    logger.info(f"メインタスクを開始")

    from verl.utils.fs import copy_local_path_from_hdfs
    from transformers import AutoTokenizer

    # 1) 初期化

    # 1-1) 設定を表示
    from pprint import pprint
    from omegaconf import OmegaConf
    # pprint(OmegaConf.to_container(config, resolve=True))
    OmegaConf.resolve(config)
    logger.debug(f"設定: {config}")

    # 1-2) Hadoop File System (HDFS) からチェックポイントをダウンロード
    local_path = copy_local_path_from_hdfs(config.actor_rollout_ref.model.path)
    logger.debug(f"モデルのローカルパス: {local_path}")

    # 1-3) トークナイザーの初期化
    from verl.utils import hf_tokenizer
    tokenizer = hf_tokenizer(local_path)
    logger.debug(f"トークナイザー: {tokenizer}")

    # 1-4) Rayのワーカーグループの初期化
    if config.actor_rollout_ref.actor.strategy == 'fsdp':
        logger.debug("FSDPワーカーグループを使用")
        assert config.actor_rollout_ref.actor.strategy == config.critic.strategy
        from verl.workers.fsdp_workers import ActorRolloutRefWorker, CriticWorker
        from verl.single_controller.ray import RayWorkerGroup
        ray_worker_group_cls = RayWorkerGroup

    elif config.actor_rollout_ref.actor.strategy == 'megatron':
        logger.debug("Megatronワーカーグループを使用")
        assert config.actor_rollout_ref.actor.strategy == config.critic.strategy
        from verl.workers.megatron_workers import ActorRolloutRefWorker, CriticWorker
        from verl.single_controller.ray.megatron import NVMegatronRayWorkerGroup
        ray_worker_group_cls = NVMegatronRayWorkerGroup

    else:
        raise NotImplementedError

    from verl.trainer.ppo.ray_trainer import ResourcePoolManager, Role

    role_worker_mapping = {
        Role.ActorRollout: ray.remote(ActorRolloutRefWorker),
        Role.Critic: ray.remote(CriticWorker),
        Role.RefPolicy: ray.remote(ActorRolloutRefWorker)
    }

    global_pool_id = 'global_pool'

    resource_pool_spec = {
        global_pool_id: [config.trainer.n_gpus_per_node] * config.trainer.nnodes,
    }

    mapping = {
        Role.ActorRollout: global_pool_id,
        Role.Critic: global_pool_id,
        Role.RefPolicy: global_pool_id,
    }

    # we should adopt a multi-source reward function here
    # - for rule-based rm, we directly call a reward score
    # - for model-based rm, we call a model
    # - for code related prompt, we send to a sandbox if there are test cases
    # - finally, we combine all the rewards together
    # - The reward type depends on the tag of the data
    if config.reward_model.enable:
        if config.reward_model.strategy == 'fsdp':
            logger.debug("FSDP報酬モデルワーカーを使用")
            from verl.workers.fsdp_workers import RewardModelWorker
        elif config.reward_model.strategy == 'megatron':
            logger.debug("Megatron報酬モデルワーカーを使用")
            from verl.workers.megatron_workers import RewardModelWorker
        else:
            raise NotImplementedError

        role_worker_mapping[Role.RewardModel] = ray.remote(RewardModelWorker)

        mapping[Role.RewardModel] = global_pool_id
    else:
        logger.debug("関数ベースの報酬マネージャーを使用")

    reward_fn = RewardManager(tokenizer=tokenizer, num_examine=0)

    # Note that we always use function-based RM for validation
    val_reward_fn = RewardManager(tokenizer=tokenizer, num_examine=1)

    resource_pool_manager = ResourcePoolManager(
        resource_pool_spec=resource_pool_spec,
        mapping=mapping
    )

    trainer = RayPPOTrainer(
        config=config,
        tokenizer=tokenizer,
        role_worker_mapping=role_worker_mapping,
        resource_pool_manager=resource_pool_manager,
        ray_worker_group_cls=ray_worker_group_cls,
        reward_fn=reward_fn,
        val_reward_fn=val_reward_fn
    )

    trainer.init_workers()

    trainer.fit()

In [ ]:
# 訓練を開始

# @hydra.main(config_path='config', config_name='ppo_trainer', version_base=None)
def main(config):

    # Rayが初期化されていない場合
    if not ray.is_initialized():
        # ローカルのRayクラスターを初期化
        logger.debug(f"Rayを初期化")
        ray.init(runtime_env={
            'env_vars': {'TOKENIZERS_PARALLELISM': 'true', 'NCCL_DEBUG': 'WARN'}
        })

    # タスクを実行
    # ray.get(main_task.remote(config))
    main_task(config)

main(ppo_trainer_config)